# Caso de Estudio: Predicción del Precio del Oro con Redes LSTM

## 1. Descubrimiento del Caso de Negocio
Contexto del proyecto: Commodity Analytics busca predecir el precio del oro (XAU/USD) utilizando datos históricos y variables macroeconómicas para optimizar decisiones de inversión.

In [ ]:
# Instalación y carga de librerías necesarias
!pip install yfinance tensorflow keras sklearn matplotlib seaborn shap keract pandas numpy

## 2. Procesamiento de Datos

In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np

# Descargar datos históricos desde Yahoo Finance
oro = yf.download("GC=F", start="2010-01-01")
dxy = yf.download("DX-Y.NYB", start="2010-01-01")
bonos = yf.download("^TNX", start="2010-01-01")
tip = yf.download("TIP", start="2010-01-01")

# Unión de datasets
df = pd.concat([oro['Close'], dxy['Close'], bonos['Close'], tip['Close']], axis=1).dropna()
df.columns = ['Oro', 'DXY', 'Bonos', 'TIP']

# Características derivadas
df['Ratio_Oro_DXY'] = df['Oro'] / df['DXY']
df['Inflacion_implicita'] = df['TIP'].pct_change()
df['Tasa_real'] = df['Bonos'] - df['Inflacion_implicita']

# Variables técnicas
df['SMA200'] = df['Oro'].rolling(window=200).mean()
df['Bollinger_up'] = df['Oro'].rolling(window=20).mean() + 2*df['Oro'].rolling(window=20).std()
df['Bollinger_down'] = df['Oro'].rolling(window=20).mean() - 2*df['Oro'].rolling(window=20).std()

df = df.dropna()
df.head()


## 3. Preparación de secuencias para LSTM

In [ ]:

from sklearn.preprocessing import RobustScaler
from keras.preprocessing.sequence import TimeseriesGenerator

scaler = RobustScaler()
data_scaled = scaler.fit_transform(df)

look_back = 90
horizon = 5

X, Y = [], []
for i in range(len(data_scaled)-look_back-horizon):
    X.append(data_scaled[i:i+look_back])
    Y.append(data_scaled[i+look_back:i+look_back+horizon, 0])  # solo precio del oro

X, Y = np.array(X), np.array(Y)
X.shape, Y.shape


## 4. División de los Datos en Train/Test

In [ ]:

split_train = int(len(X) * 0.7)
split_val = int(len(X) * 0.85)

X_train, Y_train = X[:split_train], Y[:split_train]
X_val, Y_val = X[split_train:split_val], Y[split_train:split_val]
X_test, Y_test = X[split_val:], Y[split_val:]


## 5. Modelo LSTM

In [ ]:

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(192, return_sequences=True, input_shape=(look_back, X.shape[2])),
    Dropout(0.35),
    LSTM(96),
    Dense(horizon)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


## 6. Entrenamiento del Modelo

In [ ]:

history = model.fit(X_train, Y_train, epochs=50, batch_size=32,
                    validation_data=(X_val, Y_val))


## 7. Evaluación y Visualización

In [ ]:

import matplotlib.pyplot as plt

preds = model.predict(X_test)
plt.plot(Y_test.flatten(), label='Real')
plt.plot(preds.flatten(), label='Predicción')
plt.legend()
plt.title('Predicción del Precio del Oro (Test)')
plt.show()


## 8. Interpretabilidad con SHAP (Opcional)

In [ ]:

import shap

explainer = shap.DeepExplainer(model, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

shap.summary_plot(shap_values, features=X_test[:10], feature_names=df.columns)


## 9. Guardar el Modelo

In [ ]:

model.save('modelo_oro_lstm.h5')
import joblib
joblib.dump(scaler, 'scaler_robust.pkl')
